# OCR libreria

Este ocr requiere de la compra de licencia para la libreria https://ocr.space/ se require compra de licencia que nos entrega la key que es requerida archivos con múltiples columnas

se deben definir uno de los lenguajes [Optional]
Arabic=ara
Bulgarian=bul
Chinese(Simplified)=chs
Chinese(Traditional)=cht
Croatian = hrv
Czech = cze
Danish = dan
Dutch = dut
English = eng
Finnish = fin
French = fre
German = ger
Greek = gre
Hungarian = hun
Korean = kor
Italian = ita
Japanese = jpn
Polish = pol
Portuguese = por
Russian = rus
Slovenian = slv
Spanish = spa
Swedish = swe
Turkish = tur


insert into articulos_covid values('pais','fecha','medio','item1','item2',
'item3','tipo','nombre','txt0','txtc','txtt')

In [ ]:
import ocrspace
import json
import requests
import os
from fnmatch import fnmatch
import pymssql

import time

In [ ]:
idioma='spa'
pais_extraccion= 'Spain'
apikey='tu_key'
pattern = "*.pdf"
tipo_documento='Archivo pdf'

In [ ]:


archivos_procesar=[]
arreglo=[val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk('../spain/')] for val in sublist]
# Meta comment to ease selecting text
for name in arreglo:
        if fnmatch(name, pattern):
            archivos_procesar.append(name)


print(len(arreglo))
print(len(archivos_procesar))

In [ ]:
max=20
elmaslargo=''
for name in archivos_procesar:
    cantidad= len(archivos_procesar[100].split("/"))
    frase= archivos_procesar[100].split("/")
    if cantidad <max:
        max=cantidad
        elmaslargo=frase
print(max)
print(elmaslargo)

In [ ]:
#api = ocrspace.API()
# Or if you have an API key or desired language, pass those:
#api = ocrspace.API(apikey, ocrspace.Language.Spanish)

In [ ]:
#comprime pdfs
def comprime_pdf(filename):
    arreglo=filename.split("/")
    pwd="/"
    for i in range(len(arreglo)-2):
        pwd+=  arreglo[i+1]+"/"
    
    
    variable2=pwd+"archivo.pdf"
    fila=pwd+arreglo[-1]
    variable='gs -sDEVICE=pdfwrite -dCompatibilityLevel=1.6 -dPDFSETTINGS=/ebook -dNOPAUSE -dQUIET -dBATCH -sOutputFile="{}" "{}"'.format(variable2,fila)
    #print (variable)
    os.system(variable)
    return variable2, fila

In [ ]:
#COMPRIME imagen a pdf
def comprime_pdf_tif(filename):
    arreglo=filename.split("/")
    pwd="/"
    for i in range(len(arreglo)-2):
        pwd+=  arreglo[i+1]+"/"
    
    
    variable2=pwd+"archivo.tif"
    fila=pwd+arreglo[-1]
    variable='gs -q -dNOPAUSE -r300x300 -sDEVICE=tiff24nc -sOutputFile="{}" "{}" -c quit'.format(variable2,fila)
    #print (variable)
    os.system(variable)
    return variable2, fila

In [ ]:
def comprime_pdf2(filename):#COMPRIME un docx2pdf
    arreglo=filename.split("/")
    #print(arreglo)
    pwd="/docx2pdf_temp/"
    
    #for i in range(len(arreglo)-2):
    #    pwd+=  arreglo[i+1]+"/"
    #    print(pwd)
    
    variable2=pwd+"archivo.pdf"
    #print(variable2)
    fila=pwd+arreglo[-1]
    #print(fila)
    variable='gs -sDEVICE=pdfwrite -dCompatibilityLevel=1.6 -dPDFSETTINGS=/ebook -dNOPAUSE -dQUIET -dBATCH -sOutputFile="{}" "{}"'.format(variable2,fila)
    #print (variable)
    os.system(variable)
    return variable2, fila

## Proceso de OCR archivos

In [ ]:
def ocr_space_file(filename, overlay=False, api_key='holamund', language='eng'):
    """ OCR.space API request with local file.
        Python3.5 - not tested on 2.7
    :param filename: Your file path & name.
    :param overlay: Is OCR.space overlay required in your response.
                    Defaults to False.
    :param api_key: OCR.space API key.
                    Defaults to 'helloworld'.
    :param language: Language code to be used in OCR.
                    List of available language codes can be found on https://ocr.space/OCRAPI
                    Defaults to 'en'.
    :return: Result in JSON format.
    """

    payload = {'isOverlayRequired': overlay,
               'apikey': api_key,
               'language': language
               }
    with open(filename, 'rb') as f:
        r = requests.post('https://apipro1.ocr.space/parse/image',
                          files={filename: f},
                          data=payload,
                          )
    return r.content.decode()

In [ ]:
def conectar():
    conn = pymssql.connect("xxx.xxx.xxx.xxx", "usuario_base", "clave_base", "nombre_base")
    return conn

In [ ]:
def limpia_linea(linea):
        linea = linea.replace("\n", " ")
        #linea = linea.replace("-", "")
        linea = linea.replace("\r", " ")
        return linea

In [ ]:
def limpia_guion_espacio(linea):
        linea = linea.replace("-  ", "")
        #linea = linea.replace("-", "")
        return linea

In [ ]:
print(archivos_procesar[0])

In [ ]:
tmp_arreglo= archivos_procesar

In [ ]:
print(len(tmp_arreglo))

archivos_procesar=tmp_arreglo[0:]
print(len(archivos_procesar))

In [ ]:
#prueba de retorno de OCR desde librería

retorno= ocr_space_file("/home/andres/spain/NEWSPAPERS/MAY_10/ELMUNDO/archivo.pdf",False, apikey,idioma)
jsoon_data=json.loads(retorno)
print(jsoon_data)
texto = jsoon_data['ParsedResults'][0]['ParsedText']
#texto2=limpia_linea(texto)

In [ ]:
print( archivos_procesar[0] )

In [ ]:
variable2, fila=comprime_pdf(archivos_procesar[0])
#print("/home/andresramos/ocr/chile/11 abril/biobio.cl/23 PM/Noticias/L790/L790.pdf".split("/"))
print( fila.split("/")[-1].split("_")[1])
print(variable2)

In [ ]:
print(variable2.split("/"))

In [ ]:
### PROCESAR PDFS #####

count=0
for fila2 in archivos_procesar:
        fila2=fila2.replace("’", "")
        variable2, fila=comprime_pdf(fila2)
        retorno= ocr_space_file(variable2,False, apikey,idioma)
        jsoon_data=json.loads(retorno)
        texto_extraido= ""
        for hojas in range(len(jsoon_data['ParsedResults'])) :
            texto_extraido=texto_extraido + jsoon_data['ParsedResults'][hojas]['ParsedText'] 
        texto = texto_extraido
        texto2=limpia_linea(texto)
        frase_fila= fila.split("/")
        pais= pais_extraccion
        fecha= frase_fila[5] #fila.split("/")[-1].split(".")[0]
        medio= frase_fila[6]
        if len(frase_fila)>=9:
                item1= frase_fila[6]
        else:
            item1= 'otros'
        if len(frase_fila)>=10:
                item2= frase_fila[8]
        else:
            item2= 'otros'
        item3= 'otros'
        tipo=tipo_documento 
        nombre=fila.split("/")[-1].split(".")[0]
        txt0=limpia_linea(texto)
        txtc= limpia_guion_espacio(txt0)
        txtt= 'pendiente'
        conn= conectar()
        cursor2 = conn.cursor()
        cursor2.executemany(
                        "INSERT INTO articulos_covid VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        [(pais,fecha,medio,item1,item2, item3,tipo,nombre,txt0,txtc,txtt)])
                        # you must call commit() to persist your data if you don't set autocommit to True
        conn.commit()
        cursor2.close()
        conn.close()

        if count%55==0:
            print(count)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
            time.sleep(601)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
        count+=1
  
    

In [ ]:
### TRASFORMA DOC A PDF ###
def docx2pdf (filename):
    lugar_almacenamiento ="/home/andres/docx2pdf_temp"
    filename = filename.replace(' ', '\ ')
    arreglo=filename.split("/")
    pwd="/home/andres/"
    for i in range(len(arreglo)-2):
        pwd+=  arreglo[i+1]+"/"
    #print(filename)
    #print(pwd)
    comando = 'lowriter --convert-to pdf --outdir {} {}'.format(lugar_almacenamiento,filename)
    os.system(comando)
    archivo= filename.split("/")
    nom_archivo= archivo[-1].split(".")
    #print (nom_archivo[0])
    url_archivo= lugar_almacenamiento+"/"+nom_archivo[-2]+".pdf"
    #print(url_archivo)
    return url_archivo, nom_archivo[0]
    

In [ ]:
docx2pdf(archivos_procesar[0])

In [ ]:
### PROCESAR DOCX #####

count=0
for fila2 in archivos_procesar:
        
        fila2=fila2.replace("’", "")
        fila= fila2
        fila2, nombre_archivo=docx2pdf(fila2)
        variable2, fila_no=comprime_pdf2(fila2)
        #print(variable2)
        retorno= ocr_space_file(variable2,False, apikey,idioma)
        jsoon_data=json.loads(retorno)
        texto_extraido= ""
        for hojas in range(len(jsoon_data['ParsedResults'])) :
            texto_extraido=texto_extraido + jsoon_data['ParsedResults'][hojas]['ParsedText'] 
        texto = texto_extraido
        #texto = jsoon_data['ParsedResults'][0]['ParsedText']
        texto2=limpia_linea(texto)
        frase_fila= fila.split("/")
        #print(fila)
        #print(frase_fila)
        pais= pais_extraccion
        fecha= frase_fila[4]
        medio= frase_fila[5]
        if len(frase_fila)>=9:
                item1= frase_fila[6]
        else:
            item1= 'otros'
        if len(frase_fila)>=10:
                item2= frase_fila[8]
        else:
            item2= 'otros'
        item3= 'otros'
        tipo=tipo_documento 
        nombre=fila.split("/")[-1].split(".")[0]
        txt0=limpia_linea(texto)
        txtc= limpia_guion_espacio(txt0)
        txtt= 'pendiente'
        conn= conectar()
        cursor2 = conn.cursor()
        cursor2.executemany(
                        "INSERT INTO articulos_covid VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        [(pais,fecha,medio,item1,item2, item3,tipo,nombre_archivo,txt0,txtc,txtt)])
                        # you must call commit() to persist your data if you don't set autocommit to True
        conn.commit()
        cursor2.close()
        conn.close()

        if count%55==0:
            print(count)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
            time.sleep(601)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
        count+=1
  

In [ ]:
### TRANSFORMA IMAGEN A PDF

from PIL import Image
def jpg2pdf (filename):
    #print (filename)
    lugar_almacenamiento ="/jpg2pdf_tmp"
    #filename = filename.replace(' ', '\ ')
    arreglo=filename.split("/")
    pwd="/"
    for i in range(len(arreglo)-2):
        pwd+=  arreglo[i+1]+"/"
    #print(filename)
    #print(pwd)
    image1 = Image.open(filename)
    im1 = image1.convert('RGB')
    
    archivo= filename.split("/")
    nom_archivo= archivo[-1].split(".")
    #print (nom_archivo[0])
    url_archivo= lugar_almacenamiento+"/"+nom_archivo[-2]+".pdf"
    #print(url_archivo) 
    im1.save(url_archivo)
    return url_archivo, nom_archivo[0]
   

In [ ]:
def comprime_pdf_jpg(filename):
    arreglo=filename.split("/")
    #print(arreglo)
    pwd="/jpg2pdf_tmp/"
    
    #for i in range(len(arreglo)-2):
    #    pwd+=  arreglo[i+1]+"/"
    #    print(pwd)
    
    variable2=pwd+"archivo.pdf"
    #print(variable2)
    fila=pwd+arreglo[-1]
    #print(fila)
    variable='gs -sDEVICE=pdfwrite -dCompatibilityLevel=1.6 -dPDFSETTINGS=/ebook -dNOPAUSE -dQUIET -dBATCH -sOutputFile="{}" "{}"'.format(variable2,fila)
    #print (variable)
    os.system(variable)
    return variable2, fila

In [ ]:
### PROCESAR JPG #####

count=0
for fila2 in archivos_procesar:
        fila2=fila2.replace("’", "")
        fila= fila2
        fila2, nombre_archivo=jpg2pdf(fila2)
        variable2, fila_no=comprime_pdf_jpg(fila2)
        #print(variable2)
        retorno= ocr_space_file(variable2,False, apikey,idioma)
        jsoon_data=json.loads(retorno)
        texto_extraido= ""
        for hojas in range(len(jsoon_data['ParsedResults'])) :
            texto_extraido=texto_extraido + jsoon_data['ParsedResults'][hojas]['ParsedText'] 
        texto = texto_extraido
        texto2=limpia_linea(texto)
        frase_fila= fila.split("/")
        #print(frase_fila)
        pais= pais_extraccion
        fecha=  fila.split("/")[-1].split(".")[0].split("_")[1] #frase_fila[6]
        #print(fecha)
        medio= frase_fila[2]
        if len(frase_fila)>=9:
                item1= frase_fila[6]
        else:
            item1= 'otros'
        if len(frase_fila)>=10:
                item2= frase_fila[8]
        else:
            item2= 'otros'
        item3= 'otros'
        tipo=tipo_documento 
        nombre=fila.split("/")[-1].split(".")[0]
        txt0=limpia_linea(texto)
        txtc= limpia_guion_espacio(txt0)
        txtt= 'pendiente'
        conn= conectar()
        cursor2 = conn.cursor()
        cursor2.executemany(
                        "INSERT INTO articulos_covid VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        [(pais,fecha,medio,item1,item2, item3,tipo,nombre,txt0,txtc,txtt)])
                        # you must call commit() to persist your data if you don't set autocommit to True
        conn.commit()
        cursor2.close()
        conn.close()

        if count%55==0:
            print(count)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
            time.sleep(601)
            localtime = time.localtime()
            print(time.strftime("%I:%M:%S %p", localtime))
        count+=1
  
    